In [15]:
import os
import numpy as np
import torch as T
import torch.nn as nn
import torch.optim as optim

from recurrent_network import RecurrentNetwork

In [16]:
'''
Testing for any forward pass / backpropagation errors
'''

# ssm config
embedding_size = 64
state_space_size = 256
input_size = 4
output_size = input_size
batch_size = 1
seq_len = 10000

#device = T.device("cuda" if T.cuda.is_available() else "cpu")
device = "cuda"
print(device)

# instantiate modepl
model = RecurrentNetwork(
    embedding_size=embedding_size,
    state_space_size=state_space_size,
    input_size=input_size,
    output_size=output_size,
    batch_size=batch_size,
    chkpt_dir='temp/',
    device=device
)
# backprop setup
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# generate input and target data
inputs = T.randn(batch_size, seq_len, input_size, dtype=T.float32, device=device)
targets = T.randn(batch_size, seq_len, input_size, dtype=T.float32, device=device)

# training loop for 1 step to test backprop
#model.train()

cuda


In [17]:
# forward pass
h_t = model.get_inital_hidden_state()
y_t, h_t = model(inputs, h_t)
preds = model.pred(y_t)
loss = criterion(preds, targets)

In [18]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

print("Loss:", loss.item())
print("Backpropagation and optimizer step completed successfully.")

Loss: 1.2844908237457275
Backpropagation and optimizer step completed successfully.


In [19]:
'''print(device)'''

'print(device)'

In [20]:
# checking for numerical stability
print(preds[-1], h_t[-1])

tensor([[ 0.1774,  0.3512,  0.2284, -0.9344],
        [ 0.1359,  0.3805,  0.2504, -0.9420],
        [ 0.1771,  0.3693,  0.2579, -0.9532],
        ...,
        [ 0.2815,  0.2287,  0.3277, -0.9483],
        [ 0.2186,  0.3457,  0.2527, -0.9306],
        [ 0.1763,  0.3587,  0.2286, -0.9314]], device='cuda:0',
       grad_fn=<SelectBackward0>) tensor([[ 1.5187e-04-8.7457e-05j, -2.7382e-04-5.3371e-05j,
          3.6742e-04-4.5564e-04j,  ...,
          2.1592e-04-1.3731e-04j, -2.9898e-04-4.4613e-06j,
          2.6302e-04+1.3385e-04j],
        [ 7.4134e-05+4.7742e-04j,  2.2923e-04+7.6702e-05j,
          9.0115e-05+7.5115e-04j,  ...,
         -3.6614e-04+2.9367e-04j,  1.9932e-04-8.1488e-04j,
         -5.9819e-04+3.2334e-04j],
        [-2.5943e-04+1.3169e-04j, -2.2749e-04+1.3137e-04j,
         -3.7131e-04+1.5108e-04j,  ...,
         -1.3661e-04-3.8630e-04j,  4.9617e-04+7.4156e-05j,
         -2.6413e-04-4.5138e-04j],
        ...,
        [-8.9820e-05+1.6072e-04j, -6.4200e-05+4.0081e-06j,
        

In [21]:
import torch as T

def count_nans(y_t):
    """
    Counts the number of NaN values in a tensor.
    
    Args:
        y_t (torch.Tensor): Input tensor of any shape.
        
    Returns:
        int: Number of NaN values in the tensor.
    """
    return T.isnan(y_t).sum().item()


count_nans(preds)

0